In [1]:
# organize imports
import cv2
import imutils
import numpy as np

# global variables
bg = None

In [2]:
def run_avg(image, aWeight):
    global bg
    # initialize the background
    if bg is None:
        bg = image.copy().astype("float")
        return

    # compute weighted average,accumulate it and update the background
    cv2.accumulateWeighted(image, bg, aWeight)

In [3]:
def segment(image, threshold=25):
    global bg
    # find the absolute difference between background and current frame
    diff = cv2.absdiff(bg.astype("uint8"), image)

    # threshold the diff image so that we get the foreground
    thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)[1]

    # get the contours in the thresholded image
    cnts, hierarchy = cv2.findContours(thresholded.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    

    # return None, if no contours detected
    if len(cnts) == 0:
        return
    else:
        # based on contour area, get the maximum contour which is the hand
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)

In [11]:
if __name__ == "__main__":
    # initialize weight for running average
    aWeight = 0.5
    fr = []#creating an empty list named fr
    fr_pre=[]
    # get the reference to the webcam
    camera = cv2.VideoCapture(0)

    # region of interest (ROI) coordinates
    top, right, bottom, left = 20, 350, 225, 590

    # initialize num of frames
    num_frames = 0

    # keep looping, until interrupted
    while(True):
        # get the current frame
        (grabbed, frame) = camera.read()

        # resize the frame
        frame = imutils.resize(frame, width=800)

        # flip the frame so that it is not the mirror view
        frame = cv2.flip(frame, 1)

        # clone the frame
        clone = frame.copy()

        # get the height and width of the frame
        (height, width) = frame.shape[:2]

        # get the ROI
        roi = frame[top:bottom, right:left]

        # convert the roi to grayscale and blur it
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        # to get the background, keep looking till a threshold is reached
        # so that our running average model gets calibrated
        if num_frames < 30:
            run_avg(gray, aWeight)
        else:
            # segment the hand region
            
            hand = segment(gray)

            # check whether hand region is segmented
            if hand is not None:
                # if yes, unpack the thresholded image and
                # segmented region
                (thresholded, segmented) = hand

                # draw the segmented region and display the frame
                #backtorgb = cv2.cvtColor(thresholded,cv2.COLOR_GRAY2RGB)
                cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))
                cv2.imshow("thresholded", thresholded)
                cv2.imshow("roi", roi)
                canned = cv2.Canny(roi,320,320)#For extrating edges we use Canny Edge detection method
                cv2.imshow("canned", canned)
                fr.append(clone)#Appending the read frame
                fr_pre.append(canned)#Appending the edge extracted frames

        # draw the segmented hand
        cv2.rectangle(clone, (left, top), (right, bottom), (0,255,0), 2)

        # increment the number of frames
        num_frames += 1

        # display the frame with segmented hand
        cv2.imshow("Video Feed", clone)

        # observe the keypress by the user
        keypress = cv2.waitKey(1) & 0xFF

        # if the user pressed "q", then stop looping
        if keypress == ord("q"):
            break

# free up memory
camera.release()
cv2.destroyAllWindows()

In [4]:
def framing():
    # initialize weight for running average
    aWeight = 0.5
    fr = []#creating an empty list named fr
    fr_pre=[]
    # get the reference to the webcam
    camera = cv2.VideoCapture(0)

    # region of interest (ROI) coordinates
    top, right, bottom, left = 10, 350, 225, 590

    # initialize num of frames
    num_frames = 0

    # keep looping, until interrupted
    while(True):
        # get the current frame
        (grabbed, frame) = camera.read()

        # resize the frame
        frame = imutils.resize(frame, width=800)

        # flip the frame so that it is not the mirror view
        frame = cv2.flip(frame, 1)

        # clone the frame
        clone = frame.copy()

        # get the height and width of the frame
        (height, width) = frame.shape[:2]

        # get the ROI
        roi = frame[top:bottom, right:left]

        # convert the roi to grayscale and blur it
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        # to get the background, keep looking till a threshold is reached
        # so that our running average model gets calibrated
        if num_frames < 30:
            run_avg(gray, aWeight)
        else:
            # segment the hand region
            hand = segment(gray)

            # check whether hand region is segmented
            if hand is not None:
                # if yes, unpack the thresholded image and
                # segmented region
                (thresholded, segmented) = hand

                # draw the segmented region and display the frame
                cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))
                cv2.imshow("Thesholded", thresholded)
                canned = cv2.Canny(thresholded,320,320)#For extrating edges we use Canny Edge detection method
                
                fr.append(thresholded)#Appending the read frame
                fr_pre.append(canned)#Appending the edge extracted frames

        # draw the segmented hand
        cv2.rectangle(clone, (left, top), (right, bottom), (0,255,0), 2)

        # increment the number of frames
        num_frames += 1

        # display the frame with segmented hand
        cv2.imshow("Video Feed", clone)

        # observe the keypress by the user
        keypress = cv2.waitKey(1) & 0xFF

        # if the user pressed "q", then stop looping
        if keypress == ord("q"):
            break

    # free up memory
    camera.release()
    cv2.destroyAllWindows()
    return fr_pre,fr

In [5]:
frames,ogframes = framing()#calling function framing & then extracting the images         
cv2.destroyAllWindows()

In [6]:
diff = []#creatin a list variable
for i in range(0,len(frames)-1):#defining the range
    print(frames[i],frames[i+1])#checking the frames presence
    diff.append(cv2.absdiff(frames[i],frames[i+1]))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [

In [7]:
#Each image has 3 arrays namely for R(Red),G(Green) & B(Blue) color defining the color of the image, we need to calculate the mean among those & hence we call it intra-array mean
np.mean(diff)#intra-array mean can be simply calculated using this

7.847273156430133

In [8]:
mn = np.mean(diff)#This gives mean
st_d = np.std(diff)#This gives standard deviation
print(mn,st_d)#we check the mean & standard deviation

7.847273156430133 44.039470465686335


In [9]:
a = 4#Setting a random value we can modify it to any value 
ts = mn + (a * st_d)#defining the standard threshold value for the project/global threshold value
print('The threshold==>',ts)

The threshold==> 184.00515501917548


In [10]:
a_fr = []#Creating an empty list
for i in range(len(diff)):#Defining the for loop to be looped over all the frames obtained after finding the frames resulted from subtracting
    mn = np.mean(diff[i])#Calculating the mean for each frame
    st_d = np.std(diff[i])#Calculating the standard deviation for each frame
    fr_ts = mn + (4*st_d)#Finding the threshold values for each frame/image
    print(i,fr_ts)
    a_fr.append([i,fr_ts])#Appending the frame number & the threshold values

0 13.514200772045541
1 13.514200772045541
2 9.000018594393694
3 10.064838021342556
4 21.165602738755567
5 101.55508620918734
6 153.4694251719147
7 166.53115781165735
8 168.52612740164997
9 155.4210838283432
10 146.95172508818973
11 139.29332827077764
12 138.52032032261855
13 135.93906744783092
14 143.54724718014185
15 144.66598786397438
16 140.0621832545814
17 128.97673523036025
18 122.24296855805441
19 115.40676455422918
20 120.74638084686826
21 124.3260661844275
22 123.72200802590496
23 120.74638084686825
24 128.14334094887562
25 132.90638586641214
26 151.2799359591968
27 158.57629657683842
28 169.92790939325795
29 165.88257999386408
30 149.13116796289006
31 154.09938245648974
32 163.06478485928238
33 150.20930648580094
34 134.5108180544989
35 129.3086332878432
36 125.35502772596985
37 129.1427877390737
38 127.47281974275676
39 134.1914229927779
40 135.46461092060707
41 144.36848345822597
42 143.92110270246732
43 144.66598786397438
44 147.75449823495052
45 164.70880728497949
46 171.0

In [11]:
imp_fr = []#Creating an empty list
for i,ac_tr in(a_fr):#Defining the loop on the list obtained from above code
    if ac_tr >= ts:#Comapring the threshold values to the standard threshold/global threshold values
        print(i,ac_tr)
        imp_fr.append([i,ac_tr])#Appending the list with the imp frames based on their index & the values

47 184.24435794602573
50 189.1007676759251
51 206.81802514541727
52 215.0503573088425
53 224.41825104426536
54 227.42146552698384
55 217.59209026027443
56 200.5672566859202
57 185.87720574347426
58 206.03405686316364
59 209.61574596232475
60 212.1220544396177
61 198.24007116518146
62 195.33336315488054
63 204.40422623789792
64 225.3761134470447
65 238.05386758116188
66 245.12407138487842
67 246.2163974274595
68 242.4371092888583
69 241.148698426193
70 238.73021826122815
71 240.43476003262006
72 245.64902487342647
73 240.3453619146008
74 231.45628452146826
76 200.62105422079105
77 208.6873929986388
78 212.32532499542086
79 235.51094960631102
80 247.60718240214712
81 249.03286443237516
82 248.16984523516732
83 242.9681533530434
84 239.35968119717106
85 222.68343093265656
86 212.93395344082217
87 203.7168303580547
88 196.10537410745223
89 195.11222609418368
90 190.9184236464751
91 196.87434870669577
92 205.14189845800107
93 197.257707961054
94 192.2142563874436
95 205.29962281881168
96 20

In [12]:
key_fr = []#Creating an empty list
for i,_ in imp_fr:#Defining the loop over the list obtained from above code
    key_fr.append(ogframes[i])#This extracts the frames based on the index of frames 
    print(diff[i],i)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] 47
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] 50
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] 51
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] 52
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] 53
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] 54
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] 55
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] 56
[[0 0 0 ... 0 0 

In [13]:
for i in range(len(key_fr)):
    cv2.imshow('keys',key_fr[i])
    print("Frame no==>",i)
    k = cv2.waitKey(100) & 0xFF
    if k == ord('q'):
        break
cv2.destroyAllWindows()

Frame no==> 0
Frame no==> 1
Frame no==> 2
Frame no==> 3
Frame no==> 4
Frame no==> 5
Frame no==> 6
Frame no==> 7
Frame no==> 8
Frame no==> 9
Frame no==> 10
Frame no==> 11
Frame no==> 12
Frame no==> 13
Frame no==> 14
Frame no==> 15
Frame no==> 16
Frame no==> 17
Frame no==> 18
Frame no==> 19
Frame no==> 20
Frame no==> 21
Frame no==> 22
Frame no==> 23
Frame no==> 24
Frame no==> 25
Frame no==> 26
Frame no==> 27
Frame no==> 28
Frame no==> 29
Frame no==> 30
Frame no==> 31
Frame no==> 32


In [14]:
print(len(frames))
print(len(key_fr))

235
111


In [15]:
#This function helps in writing the key-frames to a folder
for i in range(len(key_fr)):
    cv2.imshow('diff',frames[i])
    cv2.imshow('keys',key_fr[i])
    str_pos = 'keyframes3\ '+str(i)+'.jpg'
    print(str_pos)
    cv2.imwrite(str_pos,key_fr[i])#Mention preferred path
    k = cv2.waitKey(100) & 0xFF
    if k == ord('q'):
        break
cv2.destroyAllWindows()

keyframes3\ 0.jpg
keyframes3\ 1.jpg
keyframes3\ 2.jpg
keyframes3\ 3.jpg
keyframes3\ 4.jpg
keyframes3\ 5.jpg
keyframes3\ 6.jpg
keyframes3\ 7.jpg
keyframes3\ 8.jpg
keyframes3\ 9.jpg
keyframes3\ 10.jpg
keyframes3\ 11.jpg
keyframes3\ 12.jpg
keyframes3\ 13.jpg
keyframes3\ 14.jpg
keyframes3\ 15.jpg
keyframes3\ 16.jpg
keyframes3\ 17.jpg
keyframes3\ 18.jpg
keyframes3\ 19.jpg
keyframes3\ 20.jpg
keyframes3\ 21.jpg
keyframes3\ 22.jpg
keyframes3\ 23.jpg
keyframes3\ 24.jpg
keyframes3\ 25.jpg
keyframes3\ 26.jpg
keyframes3\ 27.jpg
keyframes3\ 28.jpg
keyframes3\ 29.jpg
keyframes3\ 30.jpg
keyframes3\ 31.jpg
keyframes3\ 32.jpg
keyframes3\ 33.jpg
keyframes3\ 34.jpg
